In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join('..', '..')))
from src import *
from itertools import product

import numpy as np
from matplotlib import pyplot as plt
from numba import njit
import scipy.stats as stats

In [2]:
# This assumes periodic boundaries, need to fix that

def produce_data(p, L, realizations):
    data = np.zeros(L)
    for _ in range(realizations):
        ness_correlation = C_NESS(0, 0, A(L, p))
        _, eigenvectors = np.linalg.eig(ness_correlation)
        centered_probs = center_arrays(np.abs(eigenvectors.T)**2)
        data += np.mean(centered_probs, axis = 0)
    return data / realizations

In [7]:
L = 1000
ps = [0.5, 0.7, 0.9, 1.0, 1.2, 1.4, 1.7, 2.0, 3.0, 5.0]
realizations = 100

In [8]:
data = list(map(lambda p: produce_data(p, L, realizations), ps))

In [11]:
%matplotlib qt

fit_lower_bound = 20
fit_upper_bound = 800

fig, ax = plt.subplots(nrows=1, ncols=2)
grads = np.zeros(len(data))
for i, y in enumerate(data):
    x = np.abs(np.arange(L) - L/2)
    g, c = fit_log_log(x, y, np.arange(fit_lower_bound, fit_upper_bound))
    grads[i] = -g
    ax[0].plot(x, y, linewidth=2.5, label=r"$p = {}, a = {}, c = {}$".format(
        ps[i], round(-g, 3), round(math.exp(c)), 3))
    linear_x_axis = np.arange(fit_lower_bound/2, fit_upper_bound/2)
    ax[0].plot(linear_x_axis, math.exp(c)*(linear_x_axis**g))
ax[0].set_xlabel(r"$|i-i_\mathrm{max}|$")
ax[0].set_ylabel(r"$\left\langle|\psi(i-i_\mathrm{max})|^2\right\rangle$")
ax[0].set_yscale('log')
ax[0].set_xscale('log')
ax[0].set_title(r'$L = 1000, \mathrm{realizations} = 100, y(x) \approx cx^a$')
ax[0].legend()

ax[1].plot(ps, grads, label=r"$y(p)=a(p)$")
ax[1].plot(ps, ps, label=r"$y(p)=p$")
ax[1].legend()

plt.show()
